<a href="https://colab.research.google.com/github/srijit43/NLP-theory-and-code/blob/main/05_11_BestDR_vs_BestGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Machine Learning Classifiers: Model selection

### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Split into train/test

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['body_text', 'body_len', 'punct%']], data['label'], test_size=0.2)

### Vectorize text

In [3]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

###Adjustment

In [11]:
#tfidf_vect_fit = tfidf_vect_fit.rename(str,axis='columns')


tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])  #will have same cols as test and train as it was fitted on train data only recognizes that
tfidf_test = tfidf_vect_fit.transform(X_test['body_text'])

X_train_vect = pd.concat([X_train[['body_len','punct%']].reset_index(drop=True),pd.DataFrame(tfidf_train.toarray())],axis=1)
X_test_vect =  pd.concat([X_test[['body_len','punct%']].reset_index(drop=True),pd.DataFrame(tfidf_test.toarray())],axis=1)

X_train_vect = X_train_vect.rename(str,axis=1)
X_test_vect = X_test_vect.rename(str,axis=1)
#y_train = y_train.rename(str,axis=1)
#y_test = y_test.rename(str,axis=1)

In [12]:
print(X_train_vect.head())
print(X_test_vect.head())

   body_len  punct%    0    1    2    3    4    5    6    7  ...  7113  7114  \
0        37     5.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1       128     4.7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
2        62     8.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
3        45    11.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4       120     4.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   

   7115  7116  7117  7118  7119  7120  7121  7122  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 7125 columns]
   body_len  punct%    0    1    2    3    4    5    6    7  ...  7113  7114  \
0        40     2.5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1        16    12.5  0

### Final evaluation of models

In [13]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [14]:
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

start = time.time()   #start time

rf_model = rf.fit(X_train_vect, y_train)

end = time.time()

fit_time = (end - start) #fit time

start = time.time()

y_pred = rf_model.predict(X_test_vect)

end = time.time()

pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit time:{} , Predict time:{} , Precision: {} / Recall: {} / Accuracy: {}'.format(round(fit_time,3),round(pred_time,3),
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time:11.145 , Predict time:0.194 , Precision: 1.0 / Recall: 0.794 / Accuracy: 0.974


In [15]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)


start = time.time()
y_pred = gb_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit time:{}, Pred_time:{}, Precision: {} / Recall: {} / Accuracy: {}'.format(round(fit_time,3),round(pred_time,3),
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time:213.341, Pred_time:0.131, Precision: 0.951 / Recall: 0.83 / Accuracy: 0.973
